In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt 
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model,load_model,Sequential
from tensorflow.keras.preprocessing.image import load_img,save_img,img_to_array
from tensorflow.keras.layers import Dense, Dropout, Flatten,Conv2D,MaxPooling2D,BatchNormalization
from keras.callbacks import ModelCheckpoint
from tensorflow import keras
from keras import applications
from tensorflow.keras.layers import Dense, MaxPool2D, Dropout, Flatten

In [ ]:
batch = 128
num_classes = 200
epochs = 10
input_shape = (64,64,3)

In [ ]:
model = Sequential()
model.add(Conv2D(32,kernel_size=(5,5),padding= 'same',activation = 'relu',input_shape = input_shape,kernel_regularizer='l2'))
model.add(Conv2D(32,kernel_size=(5,5),padding= 'same',activation = 'relu',kernel_regularizer='l2'))
model.add(BatchNormalization())    
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(64,kernel_size=(3,3),padding= 'same',activation = 'relu',kernel_regularizer='l2'))
model.add(Conv2D(64,kernel_size=(3,3),padding= 'same',activation = 'relu',kernel_regularizer='l2'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(64,kernel_size=(3,3),padding= 'same',activation = 'relu',kernel_regularizer='l2'))
model.add(Conv2D(128,kernel_size=(3,3),padding= 'same',activation = 'relu',kernel_regularizer='l2'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(512,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes,activation='softmax'))
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=3e-4,
    decay_steps=10000,
    decay_rate=0.9)
optimizer = keras.optimizers.Adam(learning_rate=lr_schedule)
model.compile(loss=keras.losses.categorical_crossentropy,optimizer=optimizer,metrics=["accuracy"])

In [ ]:
model.summary()

In [ ]:
data = ImageDataGenerator(rescale = 1./255,horizontal_flip=True,width_shift_range = 0.3,height_shift_range=0.3)
traindata = data.flow_from_directory('/kaggle/input/image-detect/train',target_size=(64,64),batch_size=batch,class_mode = "categorical",shuffle = True)

In [ ]:
val_txt = pd.read_csv('/kaggle/input/image-detect/val/val_annotations.txt',sep='\t',header= None) # replace with your path
val_txt = val_txt.drop(val_txt.columns[[2,3,4,5]],axis =1)
val_df = pd.DataFrame(data=val_txt)
data_ = ImageDataGenerator(rescale = 1./255)
val_data = data_.flow_from_dataframe(val_df,directory = '/kaggle/input/image-detect/val/images',x_col = val_df.columns[0],y_col=val_df.columns[1],target_size = (64,64),color_mode="rgb",class_mode="categorical")

In [ ]:
his = model.fit(traindata , validation_data = val_data,epochs = 25)

In [ ]:

print(his.history.keys())
# summarize history for accuracy
plt.plot(his.history['accuracy'])
plt.plot(his.history['val_accuracy'])
plt.title('scratch_model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()
model.save('model6.h5')

In [ ]:
data = ImageDataGenerator(rescale = 1./255,rotation_range=20,horizontal_flip=True,width_shift_range = 0.3,height_shift_range=0.3)
traindata = data.flow_from_directory('/kaggle/input/image-detect/train',target_size=(128,128),batch_size=batch,class_mode = "categorical",shuffle = True)

In [ ]:
val_data = data_.flow_from_dataframe(val_df,directory = '/kaggle/input/image-detect/val/images',x_col = val_df.columns[0],y_col=val_df.columns[1],target_size = (128,128),color_mode="rgb",class_mode="categorical")

In [ ]:
filepath="weights-improvement-{epoch:02d}-{val_accuracy:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [ ]:
from tensorflow.keras.layers import GlobalAveragePooling2D
base_model = applications.VGG16(include_top=False, weights='imagenet',input_shape=(128,128,3)) 


x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
predictions = Dense(200, activation='softmax')(x)

#print(len(base_model.layers))
model = Model(inputs=base_model.input, outputs=predictions)


for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer='rmsprop', loss='categorical_crossentropy',metrics=["accuracy"])

In [ ]:
history = model.fit(traindata,validation_data=val_data,epochs=35,callbacks = callbacks_list)

In [ ]:
print(history.history.keys())  
# summarize history for accuracy
plt.plot(history.history['accuracy']) 
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
model.save('model7.h5')

In [ ]:
model = load_model('./model7.h5')
model.summary()